# Tải các thư viện cho việc cào dữ liệu

In [1]:
!pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00


In [2]:
!apt-get update
!apt-get install chromium chromium-driver

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [810 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,082 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [2,242 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,695 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [51.1 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,975 kB]
Get:1

# Import thư viện

In [1]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import csv
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import concurrent.futures
import time
from multiprocessing import Pool

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Tối ưu Chrome để phù hợp cho việc vào web

In [7]:
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

In [ ]:
driver = web_driver()
driver.get('https://www.kidsplaza.vn/')
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

## Lấy đường link của trang web và sản phẩm sau đó lưu lại 

In [ ]:
import re
a = []
for i in range(1,12):
    a.append(f"level0 nav-{i} category-item level-top parent ui-menu-item")

In [ ]:
a

['level0 nav-1 category-item level-top parent ui-menu-item',
 'level0 nav-2 category-item level-top parent ui-menu-item',
 'level0 nav-3 category-item level-top parent ui-menu-item',
 'level0 nav-4 category-item level-top parent ui-menu-item',
 'level0 nav-5 category-item level-top parent ui-menu-item',
 'level0 nav-6 category-item level-top parent ui-menu-item',
 'level0 nav-7 category-item level-top parent ui-menu-item',
 'level0 nav-8 category-item level-top parent ui-menu-item',
 'level0 nav-9 category-item level-top parent ui-menu-item',
 'level0 nav-10 category-item level-top parent ui-menu-item',
 'level0 nav-11 category-item level-top parent ui-menu-item']

In [ ]:
menu_items = soup.find_all('li', class_= 'level0 nav-1 category-item first level-top parent ui-menu-item') #a
urls = [item.a['href'] for item in menu_items if item.a]

In [ ]:
urls

['https://www.kidsplaza.vn/me-bau-va-sau-sinh.html']

In [ ]:
# Lưu tất cả url vào một file csv
csv_file_path = 'Kid_plaza.csv'
with open(csv_file_path, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['URL'])
    for url in urls:
        csv_writer.writerow([url])

In [ ]:
csv_file_path = 'Kid_plaza.csv'
with open(csv_file_path, 'a', newline='') as csv_file:
    for url in urls:
        csv_file.write(url)

In [ ]:
url_df = pd.read_csv('Kid_plaza.csv')
url_df.iterrows()

<generator object DataFrame.iterrows at 0x000001787BC99140>

In [ ]:
for index, row in url_df.iterrows():
    url = row['URL']
    print(url)
    response = requests.get(url)

https://www.kidsplaza.vn/sua-va-thuc-pham.html
https://www.kidsplaza.vn/thuc-pham-dinh-duong.html
https://www.kidsplaza.vn/bim-ta-va-ve-sinh.html
https://www.kidsplaza.vn/binh-sua-va-phu-kien.html
https://www.kidsplaza.vn/do-so-sinh.html
https://www.kidsplaza.vn/thoi-trang-va-phu-kien.html
https://www.kidsplaza.vn/vitamin-va-suc-khoe.html
https://www.kidsplaza.vn/do-dung-an-uong.html
https://www.kidsplaza.vn/giat-xa-va-tam-goi.html
https://www.kidsplaza.vn/do-choi-va-hoc-tap.html
https://www.kidsplaza.vn/me-bau-va-sau-sinh.html


In [9]:
url_df = pd.read_csv('Kid_plaza.csv')
all_product_links = []
for index, row in url_df.iterrows():
    url = row['URL']
    while True:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        product_links = [link['href'] for link in soup.find_all('a', class_='product photo product-item-photo')]
        all_product_links.extend(product_links)

        next_page_link = soup.find('li', class_='item pages-item-next')
        if next_page_link:
            url = next_page_link.a['href']
        else:
            break

In [ ]:
len(all_product_links)

4686

In [ ]:
all_product_links

['https://www.kidsplaza.vn/sua-nan-optipro-plus-so-4-5hmo-800g-cho-be-2-6-tuoi.html',
 'https://www.kidsplaza.vn/nuoc-pha-sua-aoi-1-5l.html',
 'https://www.kidsplaza.vn/thung-sua-nuoc-nan-grow-4-180ml-cho-tre-tu-12-thang-1-thung-6-loc.html',
 'https://www.kidsplaza.vn/thung-sua-nuoc-nan-grow-4-110ml-cho-tre-tu-12-thang-1-thung-9-loc.html',
 'https://www.kidsplaza.vn/sua-nuoc-nestle-nutren-junior-rtd-4-110ml.html',
 'https://www.kidsplaza.vn/sua-bot-aptamil-duc-nutri-biotik-so-1-800g.html',
 'https://www.kidsplaza.vn/sua-bot-aptamil-duc-nutri-biotik-so-2-800g.html',
 'https://www.kidsplaza.vn/sua-bot-aptamil-duc-nutri-biotik-so-3-800g.html',
 'https://www.kidsplaza.vn/sua-bot-enfamil-enspire-s1-350g-0-12-thang.html',
 'https://www.kidsplaza.vn/sua-bot-enfagrow-enspire-s2-350g-1-3-tuoi.html',
 'https://www.kidsplaza.vn/sua-bot-enfagrow-enspire-s2-850g-1-3-tuoi.html',
 'https://www.kidsplaza.vn/sua-bot-enfagrow-enspire-s3-850g-2-6-tuoi.html',
 'https://www.kidsplaza.vn/sua-meiji-so-0-noi-

In [ ]:
csv_file_path = 'Link_KidPlaza.csv'
with open(csv_file_path, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['URL'])
    for url in all_product_links:
        csv_writer.writerow([url])

## Code dùng để lấy thông tin tên, giá và thông tin của sản phẩm

In [10]:
def scrape_product_details(product_url):
    driver = web_driver()
    driver.get(product_url)
    try:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        product_name = soup.find('h1', class_='page-title')
        product_name = product_name.text.strip() if product_name else None

        product_price = soup.find('span', class_='price')
        product_price = product_price.text.strip() if product_price else None
        table = soup.find('table', class_='data table additional-attributes')  
        if table:
            table_rows = table.find_all('tr')
            table_data = []
            for row in table_rows:
                row_data = []
                columns = row.find_all('td')
                for column in columns:
                    row_data.append(column.text.strip())
                table_data.append(row_data)
            return product_name, product_price, table_data
        else:
            return product_name, product_price, None
    finally:
        driver.quit()

In [16]:
product_urls = pd.read_csv('/content/drive/MyDrive/TTAI/Link_KidPlaza.csv')
urls = product_urls['URL'].to_list()

['https://www.kidsplaza.vn/sua-imperial-dream-xo-s4-800g.html',
 'https://www.kidsplaza.vn/sua-chua-nestle-vi-chuoi-4-100g-6m.html',
 'https://www.kidsplaza.vn/bot-an-dam-aptamil-vi-chuoi-125g.html',
 'https://www.kidsplaza.vn/bot-an-dam-aptamil-vi-ngo-125g.html',
 'https://www.kidsplaza.vn/vang-sua-nestle-vi-vani-4-100g-6m.html',
 'https://www.kidsplaza.vn/gia-vi-rac-com-tre-em-vi-ca-hoi-anpaso-35g.html',
 'https://www.kidsplaza.vn/gia-vi-rac-com-tre-em-vi-rau-cu-anpaso-35g.html',
 'https://www.kidsplaza.vn/gia-vi-rac-com-tre-em-vi-rong-bien-anpaso-35g.html',
 'https://www.kidsplaza.vn/banh-gao-lut-lusol-hinh-que-vi-tom-25g.html',
 'https://www.kidsplaza.vn/yen-sao-bestnest-kid-gio-xach-6-70ml-hang-khuyen-mai-khong-ban.html',
 'https://www.kidsplaza.vn/banh-gao-babba-s-baby-vi-tao-chuoi-45g.html',
 'https://www.kidsplaza.vn/banh-gao-babba-s-baby-vi-viet-quat-45g.html',
 'https://www.kidsplaza.vn/banh-gao-babba-s-baby-vi-dau-tao-40g.html',
 'https://www.kidsplaza.vn/banh-gao-babba-s-ba

In [1]:
product_data = []
for url in urls:
    product_name, product_price, table_data = scrape_product_details(url)
    if product_name and product_price:
        table_data = table_data if table_data else []
        product_data.append([product_name, product_price,table_data])

In [60]:
product_data

[['Sữa Imperial Dream XO s4 (800g)',
  '590.000đ',
  [['211110155'], ['Hàn Quốc'], ['1-3Y'], ['Số 4'], ['800g']]],
 ['Sữa chua Nestle vị chuối 4*100g (6M+)',
  '129.000đ',
  [['212110178'], ['Pháp'], ['6M+'], ['Vị chuối'], ['100g'], ['4 Hộp']]],
 ['Bột ăn dặm Aptamil vị chuối 125g',
  '129.000đ',
  [['213110200'], ['Anh Quốc'], ['4-6M'], ['Dạng bột'], ['125g']]],
 ['Bột ăn dặm Aptamil vị ngô 125g',
  '129.000đ',
  [['213110201'], ['Anh Quốc'], ['4-6M'], ['Dạng bột'], ['125g']]],
 ['Váng sữa Nestle vị vani 4*100g (6M+)',
  '129.000đ',
  [['212110177'], ['Pháp'], ['6M+'], ['Vị vani'], ['100g'], ['4 Hộp']]],
 ['Gia vị rắc cơm trẻ em vị cá hồi Anpaso 35g',
  '63.750đ',
  [['213170250'], ['Việt Nam'], ['12M+'], ['35g']]],
 ['Gia vị rắc cơm trẻ em vị rau củ Anpaso 35g',
  '55.250đ',
  [['213170248'], ['Việt Nam'], ['12M+'], ['35g']]],
 ['Gia vị rắc cơm trẻ em vị rong biển Anpaso 35g',
  '58.650đ',
  [['213170249'], ['Việt Nam'], ['12M+'], ['35g']]],
 ['Bánh gạo lứt Lusol hình que vị tôm 25g'

# Sau khi đã lấy được hết thông tin thì lưu lại 

In [62]:
filename = "/content/drive/MyDrive/TTAI/SanPham_Kid_plaza.csv"
with open(filename, 'w', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerows(['Product name','Product Price','Table Info'])
    writer.writerows(product_data)